In [ ]:
# Imports necesarios para la preparación de datos
import os
import shutil
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

## 1) Cargar metadatos (Folds.csv)

In [ ]:
# Ajusta la ruta del CSV si es necesario (idéntica a la usada en el notebook original)
fold_df = pd.read_csv("../input/breakhis/Folds.csv")
# Renombrar columna filename a path (mismo comportamiento original)
fold_df = fold_df.rename(columns = {"filename":"path"})
fold_df.head(3)

## 2) Extraer `filename` y `label` desde la ruta

In [ ]:
# Extraer nombre de archivo y etiqueta desde la columna `path`
fold_df['filename'] = fold_df['path'].apply(lambda x:x.split("/")[-1])
fold_df['label'] = fold_df['path'].apply(lambda x: x.split("/")[3])
fold_df.head(3)

## 3) Crear carpeta `Cancer` y copiar todas las imágenes a un único repositorio
Se respeta la lógica original: las imágenes se copian desde `../input/breakhis/BreaKHis_v1/ + path` y se renombran con `class_patientid_filename` (misma cadena que en el notebook fuente).

In [ ]:
# Crear/Resetear carpeta principal donde se agruparán todas las imágenes
ruta = "../Cancer"

if os.path.exists(ruta):
    shutil.rmtree(ruta)
os.makedirs(ruta)

# Copiar las imágenes desde la estructura original al pool ../Cancer/
for p in fold_df['path']:
    src = os.path.join("../input/breakhis/BreaKHis_v1", p)
    dest = ruta

    # Guardar los archivos con su clase y patient_id (idéntico al notebook original)
    filename = p.split("/")[3] + "_" + p.split("/")[4] + "_" + p.split("/")[-1]
    dest_path = os.path.join(dest, filename)
    shutil.copyfile(src, dest_path)

# Comprobar cantidad copiada
print('Total en ../Cancer/ :', len(os.listdir('../Cancer/')))

## 4) Construir DataFrame `df` con `file_loc`, `label` y `class` y preparar splits

In [ ]:
# Crear DataFrame `df` a partir de los archivos presentes en ../Cancer/
df = pd.DataFrame(os.listdir("../Cancer/"))
df = df.rename(columns = {0:'file_loc'})
df['label'] = df['file_loc'].apply(lambda x:x.split("_")[0])
df['class'] = df['label'].apply(lambda x: 0 if x =='benign' else 1)
df.set_index('file_loc', inplace=True)
df.head(3)

In [ ]:
# Realizar split: 10% test, 10% validación del resto, 80% entrenamiento (misma semilla que el notebook original)
data_train_and_val, data_test = train_test_split(df, test_size = 0.1, random_state = 47)
data_train, data_val = train_test_split(data_train_and_val, test_size = 0.1, random_state = 47)
print('Training size :', data_train.shape)
print('Validation size :', data_val.shape)
print('Testing size :', data_test.shape)

## 5) Balanceo del set de entrenamiento (oversampling)

In [ ]:
# Separar clases en training
train_has_cancer = data_train[data_train['class'] == 1]
train_has_no_cancer = data_train[data_train['class'] == 0]

# Upsampling de la minoría para igualar la mayoría (mismo comportamiento original)
train_has_no_cancer_upsample = resample(train_has_no_cancer, n_samples = len(train_has_cancer), 
                                     random_state = 47, replace = True)

data_train = pd.concat([train_has_cancer, train_has_no_cancer_upsample])
print('Training balanceado:', data_train['class'].value_counts().to_dict())

## 6) Crear estructura de carpetas para Train / Validation / Test y poblarlas

In [ ]:
# Lista de carpetas a crear (misma estructura que el notebook original)
folders = [
    "../Cancer_train",
    "../Cancer_test",
    "../Cancer_validation",
    "../Cancer_train/benign",
    "../Cancer_train/malignant",
    "../Cancer_validation/benign",
    "../Cancer_validation/malignant",
    "../Cancer_test/benign",
    "../Cancer_test/malignant",
]

# Borrar si existe y crear siempre (igual que el notebook original)
for folder in folders:
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.makedirs(folder)
    print('RECREADA:', folder)

print('--- TODAS LAS CARPETAS FUERON BORRADAS Y RECREADAS ---')

### 6.1) Mover imágenes del training set a sus carpetas correspondientes

In [ ]:
# Mover imágenes de entrenamiento al folder correspondiente (se reescriben nombres como image1.png, image2.png, ...)
i = 1
for img in data_train.index:
    target = df.loc[img, 'class']
    if target == 1:
        label = 'malignant'
    else:
        label = 'benign'
    src = os.path.join('../Cancer/', img)
    dest = os.path.join('../Cancer_train/', label, f'image{i}.png')
    img1 = np.array(cv2.imread(src))
    cv2.imwrite(dest, img1)
    i += 1

print('Training data movido')


### 6.2) Copiar imágenes de validación

In [ ]:
# Copiar validación
for img in data_val.index:
    target = data_val.loc[img,'class']
    if target == 1:
        label = 'malignant'
    else:
        label = 'benign'
    src = os.path.join('../Cancer/', img)
    dest = os.path.join('../Cancer_validation/', label, img)
    shutil.copyfile(src, dest)

print('Validation data lista')

### 6.3) Copiar imágenes de testing

In [ ]:
# Copiar test
for img in data_test.index:
    target = data_test.loc[img,'class']
    if target == 1:
        label = 'malignant'
    else:
        label = 'benign'
    src = os.path.join('../Cancer/', img)
    dest = os.path.join('../Cancer_test/', label, img)
    shutil.copyfile(src, dest)

print('Testing data lista')

In [ ]:
# Comprobar longitudes finales
print('#Checking their lengths')
print('Training Data:')
print(' ')
print('Benign:',len(os.listdir('../Cancer_train/benign/')))
print('Malignant::',len(os.listdir('../Cancer_train/malignant/')))
print(' ')
print('Validation Data')
print(' ')
print('Benign size:',len(os.listdir('../Cancer_validation/benign/')))
print('Malignant size :',len(os.listdir('../Cancer_validation/malignant/')))
print(' ')
print('Testing Data:')
print(' ')
print('Benign size :',len(os.listdir('../Cancer_test/benign/')))
print('Malignant size :',len(os.listdir('../Cancer_test/malignant/')))

---
Fin del notebook `DataSimplification.ipynb`.
